# Inference and Reasoning with Bayesian Networks

###### COMP4670/8600 - Introduction to Statistical Machine Learning - Assignment 2

Name: Taku Ueki

Student ID:u5934839

## Instructions

|             |Notes|
|:------------|:--|
|Maximum marks| 19|
|Weight|19% of final grade|
|Format| Complete this ipython notebook. Do not forget to fill in your name and student ID above|
|Submission mode| Use [wattle](https://wattle.anu.edu.au/)|
|Formulas| All formulas which you derive need to be explained unless you use very common mathematical facts. Picture yourself as explaining your arguments to somebody who is just learning about your assignment. With other words, do not assume that the person marking your assignment knows all the background and therefore you can just write down the formulas without any explanation. It is your task to convince the reader that you know what you are doing when you derive an argument. Typeset all formulas in $\LaTeX$.|
| Code quality | Python code should be well structured, use meaningful identifiers for variables and subroutines, and provide sufficient comments. Please refer to the examples given in the tutorials. |
| Code efficiency | An efficient implementation of an algorithm uses fast subroutines provided by the language or additional libraries. For the purpose of implementing Machine Learning algorithms in this course, that means using the appropriate data structures provided by Python and in numpy/scipy (e.g. Linear Algebra and random generators). |
| Cooperation | All assignments must be done individually. Cheating and plagiarism will be dealt with in accordance with University procedures (please see the ANU policies on [Academic Honesty and Plagiarism](http://academichonesty.anu.edu.au)). Hence, for example, code for programming assignments must not be developed in groups, nor should code be shared. You are encouraged to broadly discuss ideas, approaches and techniques with a few other students, but not at a level of detail where specific solutions or implementation issues are described by anyone. If you choose to consult with other students, you will include the names of your discussion partners for each solution. If you have any questions on this, please ask the lecturer before you act. |

$\newcommand{\dotprod}[2]{\left\langle #1, #2 \right\rangle}$
$\newcommand{\onevec}{\mathbb{1}}$

Setting up the environment (there is some hidden latex which needs to be run in this cell).

In [1]:
import itertools, copy
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Image

%matplotlib inline

## Part 1: Graphical Models

### Problem setting

We are interested to predict the outcome of the election in an imaginary country, called Under Some Assumptions (USA). There are four candidates for whom the citizens can **Vote** for: Bernie, Donald, Hillary, and Ted. The citizens live in four **Region**s: north, south, east and west. We have general demographic information about the people, namely: **Gender** (male, female) and **Hand**edness (right, left). Based on surveys done by an external company, we believe that the **Region** and **Gender** affects whether the people use their **Jacket**s full time, part time or never. Surprisingly, the company told us that the **Age** of their shoes (new, worn, old) depends on how often they wear their **Jacket**s. Furthermore, the **Gender** and their preferred **Hand** affects the **Colour** of their hat (white, black). Finally, surveys say that the citizens will **Vote** based on their **Region**, **Age** of their shoes and **Colour** of their hats.

The directed graphical model is depicted below:

In [2]:
Image(url="https://machlearn.gitlab.io/isml2017/assignments/election_model.png")

### Conditional probability tables

After paying the survey firm some more money, they provided the following conditional probability tables.

|$p(R)$ | R=n | R=s | R=e | R=w |
|:-----:|:--:|:--:|:--:|:--:|
|marginal| 0.2 | 0.1 | 0.5 | 0.2 |

|$p(G)$ | G=m | G=f |
|:-----:|:--:|:--:|
|marginal| 0.3 | 0.7 |

|$p(H)$ | H=r | H=l |
|:-----:|:--:|:--:|
|marginal| 0.9 | 0.1 |

| $p(J|R,G)$ | R=n,G=m | R=n,G=f | R=s,G=m | R=s,G=f | R=e,G=m | R=e,G=f | R=w,G=m | R=w,G=f |
|:-----:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|**J**=full $\quad$  |0.9 |0.8 |0.1 | 0.3 |0.4 |0.01| 0.02 | 0.2  |
|**J**=part $\quad$  |0.08|0.17|0.03| 0.35|0.05|0.01| 0.2  | 0.08 |
|**J**=never $\quad$ |0.02|0.03|0.87| 0.35|0.55|0.98| 0.78 | 0.72 |

| $p(A|J)$ | J=full | J=part | J=never |
|:-----:|:--:|:--:|:--:|
|**A**=new  |0.01|0.96|0.3|
|**A**=worn |0.98|0.03|0.5|
|**A**=old  |0.01|0.01|0.2|

| $p(C|G,H)$ | G=m,H=r | G=m,H=l | G=f,H=r | G=f,H=l |
|:-----:|:--:|:--:|:--:|:--:|
|**C**=black $\quad$ |0.9 |0.83 |0.17 | 0.3 |
|**C**=white $\quad$ |0.1 |0.17|0.83 | 0.7 |

The final conditional probability table is given by the matrix below. The order of the rows and columns are also given below.

In [3]:
vote_column_names = ['north,new,black', 'north,new,white', 'north,worn,black', 'north,worn,white', 
                'north,old,black', 'north,old,white', 'south,new,black', 'south,new,white', 
                'south,worn,black', 'south,worn,white', 'south,old,black', 'south,old,white', 
                'east,new,black', 'east,new,white', 'east,worn,black', 'east,worn,white', 
                'east,old,black', 'east,old,white', 'west,new,black', 'west,new,white', 
                'west,worn,black', 'west,worn,white', 'west,old,black', 'west,old,white']

vote_outcomes = ('bernie','donald','hillary','ted')

vote_pmf_array = np.array(
        [
            [0.1,0.1,0.4,0.02,0.2,0.1,0.1,0.04,0.2,0.1,0.1 ,0.1,0.4 ,0.1 ,0.1,0.1 ,0.1,0.04,0.3,0.2,0.1,0.3,0.34,0.35],
            [0.3,0.4,0.2,0.5 ,0.1,0.2,0.1,0.5 ,0.1,0.2,0.5 ,0.3,0.2 ,0.42,0.2,0.67,0.4,0.4 ,0.1,0.1,0.5,0.1,0.1 ,0.1],
            [0.5,0.4,0.3,0.3 ,0.5,0.6,0.6,0.3 ,0.5,0.4,0.36,0.3,0.28,0.3 ,0.4,0.1 ,0.4,0.16,0.4,0.2,0.3,0.3,0.4 ,0.5],
            [0.1,0.1,0.1,0.18,0.2,0.1,0.2,0.16,0.2,0.3,0.04,0.3,0.12,0.18,0.3,0.13,0.1,0.4 ,0.2,0.5,0.1,0.3,0.16,0.05],
        ]
)

The 7 conditional probability tables in are encoded in python below. 

**Base your subsequent computations on these objects**.

In [4]:
class RandomVariable(object):
    def __init__(self, name, parents, outcomes, pmf_array):
        assert isinstance(name, str)
        assert all(isinstance(_, RandomVariable) for _ in parents)
        assert isinstance(outcomes, tuple)
        assert all(isinstance(_, str) for _ in outcomes)
        assert isinstance(parents, tuple)
        assert isinstance(pmf_array, np.ndarray)
        keys = tuple(map(tuple, itertools.product(*[_.outcomes for _ in parents])))
        assert np.allclose(np.sum(pmf_array, 0), 1)
        expected_shape = (len(outcomes), len(keys))
        assert pmf_array.shape == expected_shape, (pmf_array.shape, expected_shape)
        pmfs = {k: {outcome: probability for outcome, probability in zip(outcomes, probabilities)} 
                for k, probabilities in zip(keys, pmf_array.T)}
        self.name, self.parents, self.outcomes, self.pmfs = name, parents, outcomes, pmfs


class BayesianNetwork(object):
    def __init__(self, *random_variables):
        assert all(isinstance(_, RandomVariable) for _ in random_variables)
        self.random_variables = random_variables
        

region_pmf_array = np.array([[0.2, 0.1, 0.5, 0.2]]).T
region = RandomVariable(
    name='region',
    parents=tuple(),
    outcomes=('north', 'south', 'east', 'west'), 
    pmf_array = region_pmf_array,
)

gender_pmf_array = np.array([[0.3, 0.7]]).T
gender = RandomVariable(
    name='gender',
    parents=tuple(),
    outcomes=('male', 'female'), 
    pmf_array = gender_pmf_array
)

hand_pmf_array = np.array([[0.9, 0.1]]).T
hand = RandomVariable(
    name='hand',
    parents=tuple(),
    outcomes=('left', 'right'), 
    pmf_array = hand_pmf_array
)

jacket_pmf_array = np.array(
        [
            [0.9,0.8,0.1,0.3,0.4,0.01,0.02,0.2],
            [0.08,0.17,0.03,0.35,0.05,0.01,0.2,0.08],
            [0.02,0.03,0.87,0.35,0.55,0.98,0.78,0.72],
        ]
    )
jacket = RandomVariable(
    name='jacket',
    parents=(region, gender),
    outcomes=('full', 'part', 'never'), 
    pmf_array = jacket_pmf_array
)

age_pmf_array = np.array(
        [
            [0.01,0.96,0.3],
            [0.98,0.03,0.5],
            [0.01,0.01,0.2],
        ]
    )
age = RandomVariable(
    name='age',
    parents=(jacket, ),
    outcomes=('new', 'worn', 'old'), 
    pmf_array = age_pmf_array
)

colour_pmf_array = np.array(
        [
            [0.9,0.83,0.17,0.3],
            [0.1,0.17,0.83,0.7],
        ]
    )
colour = RandomVariable(
    name='colour',
    parents=(gender, hand),
    outcomes=('black', 'white'),
    pmf_array = colour_pmf_array
)

vote = RandomVariable(
    name='vote',
    parents=(region, age, colour),
    outcomes=vote_outcomes,
    pmf_array = vote_pmf_array
)


election_model = BayesianNetwork(region, gender, hand, jacket, age, colour, vote)

### 1A (1 mark) Joint distribution of a subset

- Compute the joint distribution of **Jacket**, **Region** and **Gender**. Print in a tab separated format the resulting distribution.

In [5]:
### Solution goes here

"""to compute the joint distribution, compute p(J|R,G) * p(R) * p(G) for all possible combination of outcomes"""
sol = {}
for r in region.outcomes:
    for g in gender.outcomes:
        for j in jacket.outcomes:
            sol[r +","+ g+","+ j] = region.pmfs[region.parents][r] * gender.pmfs[gender.parents][g] * jacket.pmfs[(r,g)][j]
print(sol)



{'east,female,never': 0.34299999999999997, 'south,female,never': 0.024499999999999997, 'west,female,full': 0.027999999999999997, 'north,male,full': 0.053999999999999999, 'north,female,part': 0.023799999999999998, 'west,female,never': 0.10079999999999999, 'south,male,full': 0.0030000000000000001, 'north,male,never': 0.0011999999999999999, 'east,male,part': 0.0074999999999999997, 'west,female,part': 0.0112, 'west,male,never': 0.046800000000000001, 'south,female,full': 0.020999999999999998, 'south,male,part': 0.00089999999999999998, 'north,male,part': 0.0047999999999999996, 'north,female,full': 0.11199999999999999, 'south,male,never': 0.026099999999999998, 'north,female,never': 0.0041999999999999997, 'east,female,full': 0.0034999999999999996, 'east,male,full': 0.059999999999999998, 'east,male,never': 0.082500000000000004, 'west,male,part': 0.012, 'west,male,full': 0.0011999999999999999, 'south,female,part': 0.024499999999999997, 'east,female,part': 0.0034999999999999996}


| $p(J,R,G)$ | R=n,G=m | R=n,G=f | R=s,G=m | R=s,G=f | R=e,G=m | R=e,G=f | R=w,G=m | R=w,G=f |
|:-----:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|**J**=full $\quad$  |0.05399 |0.11199 |0.00300 | 0.02099 |0.05999 |0.00349| 0.00119 | 0.02799 |
|**J**=part $\quad$  |0.00479|0.02379|0.00089| 0.02449|0.00749|0.00349| 0.012  | 0.0112 |
|**J**=never $\quad$ |0.00119|0.00419|0.02609| 0.02449|0.08250|0.34299| 0.04680 | 0.10079 |

### 1B1 (2 marks) Variable Ordering

1. Implement a function which determines an appropriate ordering of the variables for use in the following scheme:
    - For the first node R, draw a sample from p(R).
    - For each subsequent node, draw a sample from the conditional distribution $p(X \,|\, \text{parents}(X))$ where $\text{parents}(X)$ are the parents of the variable $X$ in the graphical model.
- Use your function to compute such an ordering and print the result in a human friendly format.

In [6]:
# Solution goes here
from copy import deepcopy

def determine_order(nodes):
    _nodes = deepcopy(nodes)
    _nodes = list(_nodes)
    order = []
    
    """put the nodes that do not have any parents in the list first"""
    for node in _nodes:
        if len(node.parents) == 0:
            order.append(node)
    order_names = [n.name for n in order]

    while len(_nodes) != 0:
        for node in _nodes:
            flag = True  #if node's parents are already determined or not
            
            #remove the nodes that are already appended to the result list
            if len(node.parents) == 0:
                _nodes.remove(node)
                continue
                
            # if all parents of a node are already in the result list, put this node to the list  
            for parent in node.parents:
                if parent.name not in order_names:
                    flag = False
                    break
            if flag:
                order_names.append(node.name)
                order.append(node)
                _nodes.remove(node)
        
    return tuple(order)

print("order = ", end="")
# for o in determine_order((vote,age,hand, region, gender, colour, jacket)):
a = [region, age, colour, gender, hand, jacket, vote]
a = tuple(a)
for o in determine_order(a):
    print(o.name , end=" ")
print()
            


order = region gender hand colour jacket age vote 


### 1B2 (2 marks) Sampling

1. Given the ordering you have determined above, implement the sampler described in the previous question. If you were unable to compute the ordering in the previous question, use ``ordering = (hand, region, gender, colour, jacket, age, vote)``.
2. Draw a single sample and print the result in a human friendly format.

In [7]:
# Solution goes here
def sample_first_node(node, parents=None):
    cdf = np.cumsum([node.pmfs[node.parents][k] for k in node.outcomes])
    u = np.random.rand()
    i = np.argmax(u < cdf)
    return node.outcomes[i]


def sample_subsequent_node(node, parents):
    cdf = np.cumsum([node.pmfs[parents][k] for k in node.outcomes])
    u = np.random.rand()
    i = np.argmax(u < cdf)
    return node.outcomes[i]

print("smaple from each variable one by one")
r = sample_first_node(region)
print("region = " + r)
g = sample_first_node(gender)
print("gender = " + g)
h = sample_first_node(hand)
print("hand = " + h)
j = sample_subsequent_node(jacket, (r,g))
print("jacket = " + j)
c = sample_subsequent_node(colour, (g,h))
print("colour = " + c)
a = sample_subsequent_node(age, (j,))
print("age = " + a)
v = sample_subsequent_node(vote, (r,a,c))
print("vote = " + v)

def sampler(ordering):
    node2value = {}
    for node in ordering:
        if len(node.parents) == 0:
            node2value[node.name] = sample_first_node(node)
        else:
#             print(node.name)
#             print(tuple([node2value[parent.name] for parent in node.parents]))
            node2value[node.name] = sample_subsequent_node(node, tuple([node2value[parent.name] for parent in node.parents]))
    return node2value

print("\nsmaple from all variables")
print(sampler(determine_order((hand, region, gender, colour, jacket, age, vote))))

smaple from each variable one by one
region = north
gender = male
hand = left
jacket = full
colour = black
age = worn
vote = hillary

smaple from all variables
{'colour': 'black', 'hand': 'left', 'gender': 'male', 'age': 'worn', 'vote': 'hillary', 'region': 'west', 'jacket': 'part'}


##### 1B3 (2 marks) Marginals

1. Calculate (and show in LaTeX) the marginal distribution for **Jacket**.
- Implement a function which computes the marginal distribution of each variable. It should make use of the ordering used by your sampler.
- Implement a function which takes a list of samples the model and computes the empirical marginal distribution of each variable.
- Plot the theoretical and approximate marginals which you obtain along with the absolute percent error between the two, in the format below:

1. marginal distribution for Jacket is caluculated by
$$ P(J) = \sum_{G,R} P(J|G,R)P(G)P(R)$$

In [8]:

def determine_necessary_nodes(variable):
    """this is a function that determine necessary nodes to calculate the probability of the variable which passed as an argument"""
    result = []
    queue = list(variable.parents)
    while len(queue) != 0:
        node = queue.pop(0)
        if node not in result:
            result.append(node)
        for parent in node.parents:
            queue.append(parent)
    return determine_order(result)

def marginal(variable):
    # create a dictionary to store the result
    result = {v:0 for v in variable.outcomes}
    
    # determine the necessary nodes by using the finction above
    necessary_nodes = determine_necessary_nodes(variable)
    necessary_nodes_names = [node.name for node in necessary_nodes]
    
    # create a list that contains all outcomes of necessary nodes and also derive all possible combinations of those outcomes
    lists = [node.outcomes for node in necessary_nodes]
    combinations = list(itertools.product(*lists))
    variable_parents_names = [parent.name for parent in variable.parents]
    
    """for each combination in the combinations that I derived above, compute the probability and add it to the result dictionary"""
    for combination in combinations:
        node2prob = {}
        for node in necessary_nodes:
            parents_index = []
            for parent in node.parents:
                parents_index.append(necessary_nodes_names.index(parent.name))
            parents_attribute = tuple(combination[i] for i in parents_index)
            if node.name in variable_parents_names:
                # if the node is one of parents of target variable, multiply the node's parents probabilities
                temp = 1
                for parent in node.parents:
                    temp *= node2prob[parent.name]
                node2prob[node.name] = node.pmfs[parents_attribute][combination[necessary_nodes_names.index(node.name)]] * temp
            else:
                node2prob[node.name] = node.pmfs[parents_attribute][combination[necessary_nodes_names.index(node.name)]]
        temp = 1
        parents_index = []
        for parent in variable.parents:
            temp *= node2prob[parent.name]
            parents_index.append(necessary_nodes_names.index(parent.name))
        parents_attribute = tuple(combination[i] for i in parents_index)
        for v in variable.outcomes:
            result[v] += variable.pmfs[parents_attribute][v] * temp
    return result
            
for node in (region, gender, hand, jacket, age, colour, vote):
    print(node.name)
    print(marginal(node))


region
{'east': 0.5, 'south': 0.10000000000000001, 'north': 0.20000000000000001, 'west': 0.20000000000000001}
gender
{'male': 0.29999999999999999, 'female': 0.69999999999999996}
hand
{'right': 0.10000000000000001, 'left': 0.90000000000000002}
jacket
{'part': 0.088199999999999987, 'full': 0.28269999999999995, 'never': 0.62909999999999999}
age
{'new': 0.276229, 'old': 0.12952900000000001, 'worn': 0.59424200000000005}
colour
{'white': 0.60400000000000009, 'black': 0.39600000000000007}
vote
{'ted': 0.20465624915999994, 'bernie': 0.15481761532000005, 'donald': 0.34502085730000009, 'hillary': 0.29550527822000011}


In [9]:
def empirical_marginal_distribution(n):
    nodeName2sample = {}
    for node in (hand, region, gender, colour, jacket, age, vote):
        tmp_dic = {}
        for outcome in node.outcomes:
            tmp_dic[outcome] = 0
        nodeName2sample[node.name] = tmp_dic
    for i in range(n):
        sample = sampler(determine_order((hand, region, gender, colour, jacket, age, vote)))
        for node in (hand, region, gender, colour, jacket, age, vote):
            nodeName2sample[node.name][sample[node.name]] += 1
    return nodeName2sample
        
samples = empirical_marginal_distribution(100)
print(samples)

def plot_theoretical_and_approximate_marginals(n):
    approximate = empirical_marginal_distribution(n)
#     print(approximate)
    for node in (hand, region, gender, colour, jacket, age, vote):
        for outcome in node.outcomes:
            approximate[node.name][outcome] /= n
#     print(approximate)
    theoretical = {}
    for node in (hand, region, gender, colour, jacket, age, vote):
        theoretical[node.name] = marginal(node)
#     print(theoretical)
    for node in (hand, region, gender, colour, jacket, age, vote):
        print(node.name)
        print("outcome   exact   approx   error (%)")
        for outcome in node.outcomes:
            exact = theoretical[node.name][outcome]
            approx = approximate[node.name][outcome]
#             print(outcome + "   " + str(round(exact, 2)) + "   " + str(round(approx, 2)) + "   " + str(abs(round(exact - approx, 2))))
            print(outcome + "   " + str(exact) + "   " + str(approx) + "   " + str(abs(exact - approx)))
        print()
    
plot_theoretical_and_approximate_marginals(100)

{'colour': {'white': 54, 'black': 46}, 'hand': {'right': 9, 'left': 91}, 'gender': {'male': 31, 'female': 69}, 'age': {'new': 24, 'old': 15, 'worn': 61}, 'vote': {'ted': 21, 'bernie': 13, 'donald': 39, 'hillary': 27}, 'region': {'east': 58, 'south': 7, 'north': 22, 'west': 13}, 'jacket': {'part': 6, 'full': 33, 'never': 61}}
hand
outcome   exact   approx   error (%)
left   0.9   0.92   0.02
right   0.1   0.08   0.02

region
outcome   exact   approx   error (%)
north   0.2   0.2   0.0
south   0.1   0.09   0.01
east   0.5   0.47   0.03
west   0.2   0.24   0.04

gender
outcome   exact   approx   error (%)
male   0.3   0.37   0.07
female   0.7   0.63   0.07

colour
outcome   exact   approx   error (%)
black   0.396   0.46   0.064
white   0.604   0.54   0.064

jacket
outcome   exact   approx   error (%)
full   0.2827   0.24   0.0427
part   0.0882   0.1   0.0118
never   0.6291   0.66   0.0309

age
outcome   exact   approx   error (%)
new   0.276229   0.32   0.043771
worn   0.594242   0.6   0

### 1B4 (1 mark) Easy conditional probabilities

Compute $p(X \,|\, G=\text{female})$ for all $X$ other than $G$,
1. Approximately, using your sampler
- Exactly, using your marginal calculating function from the previous question. Hint: what happens if you set  $p(G=\text{female})=1$ in your model?
- Plot the results side by side in the same format as the previous question.
- State for which variables other than $G$ the theoretical scheme above can be used to compute such conditionals, and why.

In [11]:
# Solution goes here
"""compute p(X|G=female) by using the sampler above, the argument n is the number of the samples"""
def computeP_XgFemale_approx(n):
    nodeName2sample = {}
    for node in (hand, region, gender, colour, jacket, age, vote):
        tmp_dic = {}
        for outcome in node.outcomes:
            tmp_dic[outcome] = 0
        nodeName2sample[node.name] = tmp_dic
    num_female = 0
    for i in range(n):
        sample = sampler(determine_order((hand, region, gender, colour, jacket, age, vote)))
        if sample["gender"] == "female":
            num_female += 1
            for node in (hand, region, gender, colour, jacket, age, vote):
                nodeName2sample[node.name][sample[node.name]] += 1
    for node in (hand, region, gender, colour, jacket, age, vote):
        for outcome in node.outcomes:
            nodeName2sample[node.name][outcome] = round(nodeName2sample[node.name][outcome]/num_female,2)
    return nodeName2sample
print(computeP_XgFemale_approx(100))

"""compute theoretical p(X|G=female), by setting the probability that gender = female is 100% and using the marginal function"""
gender.pmfs[gender.parents]["male"] = 0
gender.pmfs[gender.parents]["female"] = 1

def plot_theoretical_and_approximate_marginals(n):
    approximate = computeP_XgFemale_approx(n)
    theoretical = {}
    for node in (hand, region, gender, colour, jacket, age, vote):
        theoretical[node.name] = marginal(node)
#     print(theoretical)
    for node in (hand, region, gender, colour, jacket, age, vote):
        print("p(" + node.name + " | " + "G=female)")
        print("outcome   exact   approx   error (%)")
        for outcome in node.outcomes:
            exact = theoretical[node.name][outcome]
            approx = approximate[node.name][outcome]
#             print(outcome + "   " + str(round(exact, 2)) + "   " + str(round(approx, 2)) + "   " + str(abs(round(exact - approx, 2))))
            print(outcome + "   " + str(exact) + "   " + str(approx) + "   " + str(abs(exact - approx)))
        print()

plot_theoretical_and_approximate_marginals(100)

{'colour': {'white': 0.9, 'black': 0.1}, 'hand': {'right': 0.07, 'left': 0.93}, 'gender': {'male': 0.0, 'female': 1.0}, 'age': {'new': 0.28, 'old': 0.12, 'worn': 0.6}, 'vote': {'ted': 0.19, 'bernie': 0.1, 'donald': 0.49, 'hillary': 0.22}, 'region': {'east': 0.46, 'south': 0.09, 'north': 0.25, 'west': 0.21}, 'jacket': {'part': 0.09, 'full': 0.31, 'never': 0.6}}
p(hand | G=female)
outcome   exact   approx   error (%)
left   0.9   0.9   0.0
right   0.1   0.1   0.0

p(region | G=female)
outcome   exact   approx   error (%)
north   0.2   0.19   0.01
south   0.1   0.08   0.02
east   0.5   0.46   0.04
west   0.2   0.27   0.07

p(gender | G=female)
outcome   exact   approx   error (%)
male   0   0.0   0.0
female   1   1.0   0.0

p(colour | G=female)
outcome   exact   approx   error (%)
black   0.183   0.13   0.053
white   0.817   0.87   0.053

p(jacket | G=female)
outcome   exact   approx   error (%)
full   0.235   0.24   0.005
part   0.09   0.09   1.38777878078e-17
never   0.675   0.67   0.00

4.You can use the theoretical scheme above for all variables that do not have any parents: region and hand. As these nodes do not have parents, they do not get affected by other variables. Also those variables are independent each other, so even if you change the probabilities of those variables, they do not affect on other nodes. However, if you apply the scheme to the other nodes, there will be some contradictions in the graphical model. So, that cannot be applied to those nodes.

### 1B5 (3 marks) General conditional probabilities

1. Write down the expression of the joint probability $p(R,G,H,J,A,C,V)$ in terms of the conditional probabilities in the graphical model.
- Derive $p(G = male \,|\, V = Donald)$ in terms of the conditional probabilities of the graphical model.
- Compute and display in a human friendly format the conditional distributions $p(G=g \,|\, V=v)$ for all genders $g$ and votes $v$, by naively marginalising the other variables (other than $G$ and $V$, that is).

### Solution description
$$
p(R,G,H,J,A,C,V) = p(R)p(G)p(H)p(J\,|\,R,G)p(A\,|\,J)p(C\,|\,G,H)p(V\,|\,R,A,C)
$$

\begin{equation}
\begin{split}
p(G=male \,|\, V=Donald) &= \frac{p(G=male,V=Donald)}{p(V=Donald)}
\\&= \frac{\sum_{R,H,J,A,C,V}p(R,G=male,H,J,A,C,V=Donald)}{\sum_{R,G,H,J,A,C}p(R,G,H,J,A,C,V=Donald)}
\\&= \frac{\sum_{R,H,J,A,C,V}p(R)p(G=male)p(H)p(J\,|\,R,G=male)p(A\,|\,J)p(C\,|\,G=male,H)p(V=Donald\,|\,R,A,C)}{\sum_{R,G,H,J,A,C}p(R)p(G)p(H)p(J\,|\,R,G)p(A\,|\,J)p(C\,|\,G,H)p(V=Donald\,|\,R,A,C)}
\end{split}
\end{equation}

In [12]:
import time 
start = time.time()
gender.pmfs[gender.parents]["male"] = 0.3
gender.pmfs[gender.parents]["female"] = 0.7

# Solution goes here
G_V_combinations = list(itertools.product(gender.outcomes, vote.outcomes)) 
# print(G_V_combinations[0][1])
p_V = {}
for outcome in vote.outcomes:
    p_V[outcome] = 0
p_G_V = {}
p_GgV = {}

# compute p(G, V)

for comb in G_V_combinations:
    p_G_V[comb] = 0
"""by using nested for loops create all possible combinations of all variables' outcomes and compute each probabilities"""
for r in region.outcomes:
    p_r = region.pmfs[region.parents][r]
    for g in gender.outcomes:
        p_g = gender.pmfs[gender.parents][g]
        for h in hand.outcomes:
            p_h = hand.pmfs[hand.parents][h]
            for j in jacket.outcomes:
                p_j = jacket.pmfs[(r,g)][j]
                for a in age.outcomes:
                    p_a = age.pmfs[(j,)][a]
                    for c in colour.outcomes:
                        p_c = colour.pmfs[(g,h)][c]
                        for v in vote.outcomes:
                            p_v = vote.pmfs[(r,a,c)][v]
                            p_G_V[(g,v)] += p_r*p_h*p_j*p_a*p_c*p_g*p_v
# compute p(V)                     
for r in region.outcomes:
    p_r = region.pmfs[region.parents][r]
    for g in gender.outcomes:
        p_g = gender.pmfs[gender.parents][g]
        for h in hand.outcomes:
            p_h = hand.pmfs[hand.parents][h]
            for j in jacket.outcomes:
                p_j = jacket.pmfs[(r,g)][j]
                for a in age.outcomes:
                    p_a = age.pmfs[(j,)][a]
                    for c in colour.outcomes:
                        p_c = colour.pmfs[(g,h)][c]
                        for v in vote.outcomes:
                            p_v = vote.pmfs[(r,a,c)][v]
                            p_V[v] += p_r*p_g*p_h*p_j*p_a*p_c*p_v            
# print(p_V)
# print(p_G_V)

# compute p(G | V)
for G_V_combination in G_V_combinations:
    p_GgV[G_V_combination] = p_G_V[G_V_combination]/p_V[G_V_combination[1]]
# print(p_GgV)

print("p(G=male|V=Donald) = " + str(p_GgV[("male","donald")]))
print()

print("\t ", end="")
for v in vote.outcomes:
    print(v, end="                   ")
print()
for g in gender.outcomes:
    print(g, end="  ")
    for v in vote.outcomes:
#         print(round(p_GgV[(g,v)],2), end=" ")
        print(p_GgV[(g,v)], end="  ")
    print()
print()
end = time.time()
print("run time = " + str(end - start))

p(G=male|V=Donald) = 0.214321628636

	 bernie                   donald                   hillary                   ted                   
male  0.398132990181  0.214321628636  0.363555529929  0.278437575417  
female  0.601867009819  0.785678371364  0.636444470071  0.721562424583  

run time = 0.005324125289916992


### 1B6 (2 marks) Variable elimination

Denote the graphical model consider thus far $\mathcal{M}$.

1. Derive $p(R,G,J,A,C,V)$ by marginalising over $H$ in $\mathcal{M}$. 
- Describe how the structure (connectivity) of the new graphical model (call it $\mathcal{M}_H$) over all variables other than $H$ changes after eliminating $H$ in this way.
- Describe which conditional(s) in the new graphical model differ from the original model.
- Encode the $\mathcal{M}_H$ in python similarly to $\mathcal{M}$. 

In [13]:
# Solution goes here
"""compute the joint probability p(G,H,C) first and marginalise the hand"""
GHC_combination = set()
p_GHC = {}
for g in gender.outcomes:
    p_g = gender.pmfs[gender.parents][g]
    for h in hand.outcomes:
        p_h = hand.pmfs[hand.parents][h]
        for c in colour.outcomes:
            p_c = colour.pmfs[(g,h)][c]
#             GHC_combination.add((g,h,c))
            p_GHC[(g,h,c)] = p_g * p_h * p_c

p_GC = {}
for g in gender.outcomes:
    for c in colour.outcomes:
        p_GC[(g,c)] = 0
for key in p_GHC.keys():
        p_GC[(key[0], key[2])] += p_GHC[key]
        
print(p_GC)
p_CgG = {}
for key in p_GC.keys():
    p_CgG[key] = p_GC[key]/gender.pmfs[gender.parents][key[0]]
print(p_CgG)   

#encode the new colour node. Other variables are the same as the original ones.
new_colour_pmf_array = np.array(
        [
            [0.893,0.183],
            [0.107,0.817],
        ]
    )
new_colour = RandomVariable(
    name='colour',
    parents=(gender,),
    outcomes=('black', 'white'),
    pmf_array = new_colour_pmf_array
)

# print(new_colour.pmfs[("female",)]["white"])

{('female', 'white'): 0.57190000000000007, ('male', 'white'): 0.032100000000000004, ('male', 'black'): 0.26790000000000003, ('female', 'black'): 0.12810000000000002}
{('female', 'white'): 0.81700000000000017, ('male', 'white'): 0.10700000000000001, ('male', 'black'): 0.89300000000000013, ('female', 'black'): 0.18300000000000005}


2.Hand is independent from any other nodes except for colour. Therefore, even if hand is eliminated from the graphical model, it does not affect on the structure of it. 

3.Only colour variable will be different from original model.

### 1B6 (3 marks) General estimation of conditional probabilities (revisited)

1. As you did earlier, compute and display in a human friendly format the conditional distributions $p(G=g \,|\, V=v)$ for all genders $g$ and votes $v$, by naively marginalising the other variables (other than $G$ and $V$, that is). This time however, do so using $\mathcal{M}_H$ rather than $\mathcal{M}$.
- Quantify the computational advantages of using $\mathcal{M}_H$ in this way rather than $\mathcal{M}$.
- Which variable (or variables) would be the best to eliminate in this way, in terms of the aforementioned computational advantages, and why?
- Pick a (best) variable from the previous question and call it $X$. Assuming we have eliminated $X$, which would be the "best" variable to subsequently eliminate in a similar fashion?

In [14]:
# Solution goes here

start = time.time()

vote = RandomVariable(
    name='vote',
    parents=(region, age, new_colour),
    outcomes=vote_outcomes,
    pmf_array = vote_pmf_array
)
# compute p(G, V)
"""compute new conditional distributions p(G|V) by computing with same algorithm as last question with new graphical model: without hand node"""
p_G_V = {}
for comb in G_V_combinations:
    p_G_V[comb] = 0
for r in region.outcomes:
    p_r = region.pmfs[region.parents][r]
    for g in gender.outcomes:
        p_g = gender.pmfs[gender.parents][g]
        for j in jacket.outcomes:
            p_j = jacket.pmfs[(r,g)][j]
            for a in age.outcomes:
                p_a = age.pmfs[(j,)][a]
                for c in new_colour.outcomes:
                    p_c = new_colour.pmfs[(g,)][c]
                    for v in vote.outcomes:
                        p_v = vote.pmfs[(r,a,c)][v]
                        p_G_V[(g,v)] += p_r*p_j*p_a*p_c*p_g*p_v
# compute p(V)                   
p_V = {}
for outcome in vote.outcomes:
    p_V[outcome] = 0
for r in region.outcomes:
    p_r = region.pmfs[region.parents][r]
    for g in gender.outcomes:
        p_g = gender.pmfs[gender.parents][g]
        for j in jacket.outcomes:
            p_j = jacket.pmfs[(r,g)][j]
            for a in age.outcomes:
                p_a = age.pmfs[(j,)][a]
                for c in new_colour.outcomes:
                    p_c = new_colour.pmfs[(g,)][c]
                    for v in vote.outcomes:
                        p_v = vote.pmfs[(r,a,c)][v]
                        p_V[v] += p_r*p_g*p_j*p_a*p_c*p_v



# compute p(G | V)
for G_V_combination in G_V_combinations:
    p_GgV[G_V_combination] = p_G_V[G_V_combination]/p_V[G_V_combination[1]]
# print(p_GgV)

print("\t ", end="")
for v in vote.outcomes:
    print(v, end="                   ")
print()
for g in gender.outcomes:
    print(g, end="  ")
    for v in vote.outcomes:
#         print(round(p_GgV[(g,v)],2), end=" ")
        print(p_GgV[(g,v)], end="  ")
    print()
end = time.time()
print("run time = " + str(end - start))

	 bernie                   donald                   hillary                   ted                   
male  0.398132990181  0.214321628636  0.363555529929  0.278437575417  
female  0.601867009819  0.785678371364  0.636444470071  0.721562424583  
run time = 0.0030379295349121094


2.By removing hand node from the original graphical model, we can reduce a half of the number of combinatins of other nodes' outcomes, because hand node has 2 outcomes (right and left).
Therefore, computation processes should also become half and the program gets twice faster.

3.If you think about only computational advantages, region would be the best variable to remove, because region node has the most number of outcomes which is 4. By removing region node from the graphical model, you will get quarter number of combinations of outcomes of other nodes.

4.Jacket and Age are the canidates to eliminate subsequently, because they have 3 outcomes which is the most among the remaining nodes of the graphical model. However, I assume that we are trying to predict vote by using other nodes. In this case, age is directly dependent on vote, so we should not eliminate it. Hence, jacket node would be the best node to eliminate subsequently to improve this model in terms of computational advantages.

## Part 2: Theory

### 2A (3 marks) Functions of random variables

$u$ and $v$ are independently sampled from the standard uniform distribution on the unit interval, $[0,1]$. 

If $s^2=(2u-1)^2+(2v-1)^2 \geq 1$ then $x$ is sampled from the standard normal distribution, $\mathcal{N}(0,1)$. Otherwise $x=(2u-1)\sqrt{-2 \log(s)/s}$.

How is $x$ distributed?

### Solution description
Set $2u - 1 = U$  and  $2v - 1 = V$.
<br/>
<br/>
Since $u$ and $v$ are independently sampled from standard uniform distribution, $U$ and $V$ are uniformly distributed over $(-1, 1)$.
<br/>
<br/>
Convert these two variables to polar coordinates
$$ U = r cos\theta ,  V = r sin\theta$$
Therefore, $$ r^2 = U^2 + V^2$$ 
Also the Jacobian of this conversion is $ \dfrac{dU dV}{dr d\theta} = r $.
<br/>
<br/>
When $s=(2u-1)^2+(2v-1)^2 < 1$, $x$ is calculated by $x=(2u-1)\sqrt{-2 \log(s)/s}$.
<br/>
<br/>
Set $x = rcos\theta \sqrt{\dfrac{-2 log (r^2)}{r^2}}$ and $y = rsin\theta \sqrt{\dfrac{-2 log (r^2)}{r^2}}$
<br/>
<br/>
The Jacobian of this conversion is
$ \dfrac{dxdy}{dr d\theta} = \dfrac{2}{r} $
<br/>
<br/>
According to Bishop - Pattern Recognition and Machine Learning p528, formula (11.12), $$p(x,y) = p(U,V) \begin{vmatrix} \dfrac{\partial(U,V)}{\partial(x,y)} \end{vmatrix} 
$$
Also according to Bishop - Pattern Recognition and Machine Learning p527,
<br/>
for pairs of uniformly distributed random numbers $z_1 , z_2 \in (-1,1)$ ,
<br/>
the probability that $z_1$ and $z_2$ satisfy $z_1^2 + z_2^2 \leq 1$ is expressed as $p(z_1,z_2) = \dfrac{1}{\pi}$
<br/>
Therefore, 
$p(U,V) = \dfrac{1}{\pi}$
<br/>
<br/>
$$ p(x,y) = p(U,V) \begin{vmatrix} \dfrac{\partial(U,V)}{\partial(x,y)} \end{vmatrix} = p(U,V) \dfrac{dUdV}{drd\theta} \dfrac{drd\theta}{dxdy}$$

\begin{equation}
\begin{split}
p(x,y) 
  &=
  p(U,V) \begin{vmatrix} \dfrac{\partial(U,V)}{\partial(x,y)} \end{vmatrix}
  \\&=
  p(U,V) \dfrac{dUdV}{drd\theta} \dfrac{drd\theta}{dxdy}
  \\&=
  \dfrac{1}{\pi} \times r \times \dfrac{r}{2}
  \\&=
  \dfrac{1}{\pi} \times \dfrac{r^2}{2}
\end{split}
\end{equation}
<br/>
<br/>
Since $$x = rcos\theta \sqrt{\dfrac{-2 log (r^2)}{r^2}} \text{  and  } y = rsin\theta \sqrt{\dfrac{-2 log (r^2)}{r^2}}$$
\begin{equation}
\begin{split}
x^2 + y^2 
  &=
  -2 log r^2 cos^2\theta -2 log r^2 sin^2\theta
  \\&=
  -2 log r^2
\end{split}
\end{equation}

$$ log r^2 = -\dfrac{x^2 + y^2}{2} $$

$$ r^2 = exp(-\dfrac{x^2 + y^2}{2}) $$

Plug this in $p(x,y)$ and we get
$$ p(x,y) = \dfrac{1}{2\pi} exp(-\dfrac{x^2 + y^2}{2})  $$
\begin{equation}
\begin{split}
p(x,y)
  &=
  \dfrac{1}{2\pi} exp(-\dfrac{x^2 + y^2}{2})
  \\&=
  \dfrac{1}{\sqrt{2\pi}} exp(-\dfrac{x^2}{2}) \times \dfrac{1}{\sqrt{2\pi}} exp(-\dfrac{y^2}{2})
\end{split}
\end{equation}

Hence, x's distribution is standard normal distribution. 